# Porovnání pipeline
Naší a paperu; na strukturách z paperu (apo_holo.dat)
- filtrování struktur
- určení holo/apo
- určení isoformy
- vytvoření párů na základě isoformy
- (porovnání výsledků analýz je v jiném ipynb, to jsme už vyřešili)

In [2]:
import os
from pathlib import Path

import numpy as np
import pandas as pd

from paper_dataset_utils import get_paper_apo_holo_dataframe

paper_pairs = get_paper_apo_holo_dataframe()
paper_pairs


,domain_count,ligand_codes,apo_chain_id,holo_chain_id,apo_pdb_code,holo_pdb_code
0,1,SUC;SUC;,R,P,1a0s,1a0t
1,1,SES;,A,A,1a3y,1dzj
2,1,BEZ;,B,A,1a7u,1a8u
3,2,NGA;NGA;,_,A,1a8d,1d0h
4,2,OXL;,_,A,1a8f,1ryo
...,...,...,...,...,...,...
516,1,BHC;,D,A,4pgm,1bq4
517,1,5AS;,_,A,6rhn,1rzy
518,2,NAG-MAN;,_,A,6taa,2guy
519,2,GOL;GOL;GOL;,D,B,7req,1req


## Podvybrat pouze chainy struktur, tak jako v paperu.

In [3]:
PIPELINE_OUTPUT_FILE = Path('/Users/adam/pschool/bakalarka/apo-holo-protein-structure-stats/apo_holo_structure_stats/test/test_pipeline_output/paper_dataset/filtered_chains.json')
chains = pd.read_json(PIPELINE_OUTPUT_FILE)
chains

,pdb_code,path,chain_id,is_holo,resolution,_exptl.method,sequence,uniprotkb_id,uniprot_group_size
0,1a0s,pdb_structs/1a0s.cif.gz,R,False,2.40,X-RAY DIFFRACTION,"[SER, GLY, PHE, GLU, PHE, HIS, GLY, TYR, ALA, ...",P22340,9
1,1a0t,pdb_structs/1a0t.cif.gz,P,True,2.40,X-RAY DIFFRACTION,"[SER, GLY, PHE, GLU, PHE, HIS, GLY, TYR, ALA, ...",P22340,9
2,1a30,pdb_structs/1a30.cif.gz,A,True,2.00,X-RAY DIFFRACTION,"[PRO, GLN, ILE, THR, LEU, TRP, LYS, ARG, PRO, ...",P04585,467
3,1a3l,pdb_structs/1a3l.cif.gz,L,True,1.95,X-RAY DIFFRACTION,"[ASP, ILE, VAL, LEU, THR, GLN, ALA, ALA, PHE, ...",P01837,138
4,1a3l,pdb_structs/1a3l.cif.gz,H,True,1.95,X-RAY DIFFRACTION,"[GLU, VAL, GLN, LEU, GLU, GLU, SER, GLY, PRO, ...",P01869,20
...,...,...,...,...,...,...,...,...,...
1029,5eug,pdb_structs/5eug.cif.gz,A,True,1.60,X-RAY DIFFRACTION,"[MET, ALA, ASN, GLU, LEU, THR, TRP, HIS, ASP, ...",P12295,23
1030,6rhn,pdb_structs/6rhn.cif.gz,A,False,2.15,X-RAY DIFFRACTION,"[ARG, PRO, GLY, GLY, ASP, THR, ILE, PHE, GLY, ...",P80912,9
1031,6taa,pdb_structs/6taa.cif.gz,A,False,2.10,X-RAY DIFFRACTION,"[ALA, THR, PRO, ALA, ASP, TRP, ARG, SER, GLN, ...",P0C1B3,17
1032,7req,pdb_structs/7req.cif.gz,D,False,2.20,X-RAY DIFFRACTION,"[SER, SER, THR, ASP, GLN, GLY, THR, ASN, PRO, ...",P11652,15


In [26]:

# drop duplicates, as in the paper a-h pairs there are no duplicates of (pdb_code, chain_id)
# But the outputs of the pipeline contain duplicates, as the only

chains = chains.drop_duplicates(['pdb_code', 'chain_id'])
len(chains)

1032

In [4]:


# drop duplicates, as in the paper a-h pairs there are no duplicates of (pdb_code, chain_id)
# But the outputs of the pipeline contain duplicates, as the only

# chains = chains.drop_duplicates()


# vyres to tak: pro vsechny podtrzitka najdi v outputu spravny struktury a ma byt prave 1 myslim
for i, row in enumerate(paper_pairs.itertuples()):
    row = row._asdict()  # convert to dict, so we can easily index columns with strings

    for apo_or_holo in ('apo', 'holo'):
        if row[f'{apo_or_holo}_chain_id'] != '_':
            continue

        hits = chains[chains.pdb_code == row[f'{apo_or_holo}_pdb_code']]
        if len(hits) != 1:
            print('error', row[f'{apo_or_holo}_pdb_code'])
            raise RuntimeError()

        # change '_' into actual chain id, so we can do merges with pd
        paper_pairs[f'{apo_or_holo}_chain_id'].iat[i] = hits['chain_id'].iat[0]

# concat apo and holo columns from pairs to a single `paper_chains` df
paper_chains = []
for apo_or_holo in ('apo', 'holo'):
    c1, c2 = f'{apo_or_holo}_pdb_code', f'{apo_or_holo}_chain_id'
    paper_chains.append(
        paper_pairs[[c1, c2]].rename(columns={c1: 'pdb_code', c2: 'chain_id'}))
paper_chains = pd.concat(paper_chains)
paper_chains = paper_chains.set_index(['pdb_code', 'chain_id'], verify_integrity=True)

chains = chains.merge(paper_chains, left_on=['pdb_code', 'chain_id'], right_index=True)
# todo takhlle nejde mergovat, z nejakyho duvodu mam 2 dupes
chains = chains.merge(paper_pairs, how='left', left_on=['pdb_code', 'chain_id'], right_on=['apo_pdb_code', 'apo_chain_id'])

chains = chains.merge(paper_pairs, how='left', left_on=['pdb_code', 'chain_id'], right_on=['holo_pdb_code', 'holo_chain_id'])
# drop chains that were not in paper dataset
chains = chains.dropna(subset=['apo_pdb_code_x', 'apo_pdb_code_y'], how='all')



In [5]:
chains

,pdb_code,path,chain_id,is_holo,resolution,_exptl.method,sequence,uniprotkb_id,uniprot_group_size,domain_count_x,...,apo_chain_id_x,holo_chain_id_x,apo_pdb_code_x,holo_pdb_code_x,domain_count_y,ligand_codes_y,apo_chain_id_y,holo_chain_id_y,apo_pdb_code_y,holo_pdb_code_y
0,1a0s,pdb_structs/1a0s.cif.gz,R,False,2.40,X-RAY DIFFRACTION,"[SER, GLY, PHE, GLU, PHE, HIS, GLY, TYR, ALA, ...",P22340,9,1.0,...,R,P,1a0s,1a0t,NaN,NaN,NaN,NaN,NaN,NaN
1,1a0t,pdb_structs/1a0t.cif.gz,P,True,2.40,X-RAY DIFFRACTION,"[SER, GLY, PHE, GLU, PHE, HIS, GLY, TYR, ALA, ...",P22340,9,NaN,...,NaN,NaN,NaN,NaN,1.0,SUC;SUC;,R,P,1a0s,1a0t
2,1a30,pdb_structs/1a30.cif.gz,A,True,2.00,X-RAY DIFFRACTION,"[PRO, GLN, ILE, THR, LEU, TRP, LYS, ARG, PRO, ...",P04585,467,NaN,...,NaN,NaN,NaN,NaN,1.0,GLU-ASP-LEU;,B,A,1ztz,1a30
3,1a3l,pdb_structs/1a3l.cif.gz,L,True,1.95,X-RAY DIFFRACTION,"[ASP, ILE, VAL, LEU, THR, GLN, ALA, ALA, PHE, ...",P01837,138,NaN,...,NaN,NaN,NaN,NaN,2.0,CFC;,L,L,1ruq,1a3l
4,1a3l,pdb_structs/1a3l.cif.gz,H,True,1.95,X-RAY DIFFRACTION,"[GLU, VAL, GLN, LEU, GLU, GLU, SER, GLY, PRO, ...",P01869,20,NaN,...,NaN,NaN,NaN,NaN,2.0,CFC;CFC;,H,H,1rur,1a3l
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,5eug,pdb_structs/5eug.cif.gz,A,True,1.60,X-RAY DIFFRACTION,"[MET, ALA, ASN, GLU, LEU, THR, TRP, HIS, ASP, ...",P12295,23,NaN,...,NaN,NaN,NaN,NaN,1.0,URA;,A,A,4eug,5eug
1030,6rhn,pdb_structs/6rhn.cif.gz,A,False,2.15,X-RAY DIFFRACTION,"[ARG, PRO, GLY, GLY, ASP, THR, ILE, PHE, GLY, ...",P80912,9,1.0,...,A,A,6rhn,1rzy,NaN,NaN,NaN,NaN,NaN,NaN
1031,6taa,pdb_structs/6taa.cif.gz,A,False,2.10,X-RAY DIFFRACTION,"[ALA, THR, PRO, ALA, ASP, TRP, ARG, SER, GLN, ...",P0C1B3,17,2.0,...,A,A,6taa,2guy,NaN,NaN,NaN,NaN,NaN,NaN
1032,7req,pdb_structs/7req.cif.gz,D,False,2.20,X-RAY DIFFRACTION,"[SER, SER, THR, ASP, GLN, GLY, THR, ASN, PRO, ...",P11652,15,2.0,...,D,B,7req,1req,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# remove chain dupes

##### Nekterym chybi isoform - vyprintovat a odstranit:

In [6]:
print()
print(len(chains))
print(chains[chains.isoform.isna()])
chains = chains.dropna(subset=['isoform'])  # will drop those without isoform field
print(len(chains))


1034


AttributeError: 'DataFrame' object has no attribute 'isoform'

Prvni 3 struktury jsou variable chainy imunoglobulinů (holo struktura k 1ap2 ale už má UNP).
Zbylé 3 jsou obsolete pdb struktury (ty asi nebudeme zpracovávat, takže OK).


## Porovnat klasifikaci apo, holo (jednotlive po strukturach)
- FP, FN, TP, TN
<a id='is_holo_classification'></a>

In [27]:

print('apo', np.sum(chains.apo_pdb_code_y.isna()))
print('holo', np.sum(chains.apo_pdb_code_x.isna()))
if 'is_holo_paper' not in chains.columns:
    # check, (so multiple runs of this cell are allowed)
    chains.insert(loc=3, column='is_holo_paper', value=chains.apo_pdb_code_x.isna())

# dat isholo do sklearn metrics
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(chains.is_holo_paper, chains.is_holo, digits=3))
tn, fp, fn, tp = confusion_matrix(chains.is_holo_paper, chains.is_holo).ravel()  # line from sklearn docs
print('TP+TN', tp + tn)
print('FP', fp)
print('FN', fn)


chains[chains.is_holo_paper != chains.is_holo]

apo 515
holo 517
              precision    recall  f1-score   support

       False      0.919     0.988     0.952       515
        True      0.987     0.913     0.949       517

    accuracy                          0.951      1032
   macro avg      0.953     0.951     0.951      1032
weighted avg      0.953     0.951     0.951      1032

TP+TN 981
FP 6
FN 45


,pdb_code,path,chain_id,is_holo_paper,is_holo,resolution,_exptl.method,sequence,uniprotkb_id,uniprot_group_size,...,apo_chain_id_x,holo_chain_id_x,apo_pdb_code_x,holo_pdb_code_x,domain_count_y,ligand_codes_y,apo_chain_id_y,holo_chain_id_y,apo_pdb_code_y,holo_pdb_code_y
43,1bq9,pdb_structs/1bq9.cif.gz,A,True,False,1.20,X-RAY DIFFRACTION,"[FME, ALA, LYS, TRP, VAL, CYS, LYS, ILE, CYS, ...",P24297,37,...,NaN,NaN,NaN,NaN,1.0,FME;,A,A,1bq8,1bq9
61,1c60,pdb_structs/1c60.cif.gz,A,True,False,2.00,X-RAY DIFFRACTION,"[MET, ASN, ILE, PHE, GLU, MET, LEU, ARG, ILE, ...",P00720,886,...,NaN,NaN,NaN,NaN,2.0,BME;,A,A,1c62,1c60
73,1cqf,pdb_structs/1cqf.cif.gz,A,True,False,2.20,X-RAY DIFFRACTION,"[THR, PRO, ASP, CYS, VAL, THR, GLY, LYS, VAL, ...",P69179,56,...,NaN,NaN,NaN,NaN,1.0,GAL-GLC;GAL-GLC;,E,A,1c48,1cqf
138,1ehd,pdb_structs/1ehd.cif.gz,A,False,True,1.50,X-RAY DIFFRACTION,"[PCA, ARG, CYS, GLY, SER, GLN, GLY, GLY, GLY, ...",Q9S7B3,3,...,A,A,1ehd,1ehh,NaN,NaN,NaN,NaN,NaN,NaN
144,1eoa,pdb_structs/1eoa.cif.gz,A,True,False,2.15,X-RAY DIFFRACTION,"[MET, ASN, GLY, TRP, ASN, PHE, GLN, GLU, LEU, ...",P20371,16,...,NaN,NaN,NaN,NaN,1.0,CYN;,A,A,1eo2,1eoa
145,1eoa,pdb_structs/1eoa.cif.gz,B,True,False,2.15,X-RAY DIFFRACTION,"[MET, SER, GLN, ILE, ILE, TRP, GLY, ALA, TYR, ...",P20372,16,...,NaN,NaN,NaN,NaN,1.0,CYN;,B,B,1eo2,1eoa
163,1f4v,pdb_structs/1f4v.cif.gz,A,True,False,2.22,X-RAY DIFFRACTION,"[ALA, ASP, LYS, GLU, LEU, LYS, PHE, LEU, VAL, ...",P0AE67,103,...,NaN,NaN,NaN,NaN,1.0,MET-GLY-ASP-SER-ILE-LEU-SER-GLN-ALA-GLU-ILE-AS...,A,A,1eay,1f4v
168,1f8q,pdb_structs/1f8q.cif.gz,A,True,False,2.20,X-RAY DIFFRACTION,"[ASP, VAL, SER, PHE, ARG, LEU, SER, GLY, ALA, ...",P16094,17,...,NaN,NaN,NaN,NaN,1.0,NAG-MAN-XYS;,A,A,1mri,1f8q
228,1h12,pdb_structs/1h12.cif.gz,A,True,False,1.20,X-RAY DIFFRACTION,"[ALA, PHE, ASN, ASN, ASN, PRO, SER, SER, VAL, ...",Q8RJN8,8,...,NaN,NaN,NaN,NaN,1.0,XYP-XYS;,A,A,1h13,1h12
249,1hnu,pdb_structs/1hnu.cif.gz,A,True,False,2.15,X-RAY DIFFRACTION,"[MET, SER, GLN, GLU, ILE, ARG, GLN, ASN, GLU, ...",Q05871,20,...,NaN,NaN,NaN,NaN,1.0,REO-EDO;,A,A,1hno,1hnu


##### 20 False positives (not is_holo_paper)
1vjm má taky retinal, i v LPC https://oca.weizmann.ac.il/oca-bin/Vcofc.cgi?num=1&PDB_ID=1VJM&XID=01319800001637772101
- má 15 kontaktů, z toho 10 destabilizing (hydrophil+phob)
- -> asi to odčítaj

1nmc má 5 kontaktů dle LPC https://oca.weizmann.ac.il/oca-bin/Vcofc.cgi?num=15&PDB_ID=1NMC&XID=01476500001637772518

1n13, taky dost kontaktů, ne všechny ale stabilizing https://oca.weizmann.ac.il/oca-bin/Vcofc.cgi?num=9&PDB_ID=1N13&XID=01552300001637772752


LPC s peptide ligandama nefunguje (dava 404, nebo tam peptid neni mezi ligandama, takze peptidy delali asi jinak?)

##### 23 False negatives
- mezi nima i maly molekuly < 6 atomu (ty skippuju, jak psali?!), např. kyanid (CYN)
- 5 peptidu




In [28]:
pairs = pd.read_json(Path('/Users/adam/pschool/bakalarka/apo-holo-protein-structure-stats/apo_holo_structure_stats/test/test_pipeline_output/paper_dataset/pairs.json'))
pairs

,pdb_code_apo,chain_id_apo,pdb_code_holo,chain_id_holo,lcs_result
0,2j1r,A,2j1s,A,"{'length': 151, 'i1': 0, 'i2': 0, 'mismatches'..."
1,2bnu,B,2bnq,E,"{'length': 241, 'i1': 0, 'i2': 0, 'mismatches'..."
2,2aq6,B,1y30,A,"{'length': 147, 'i1': 0, 'i2': 0, 'mismatches'..."
3,1p5h,B,1p5r,A,"{'length': 428, 'i1': 0, 'i2': 0, 'mismatches'..."
4,3ng1,B,1ng1,A,"{'length': 294, 'i1': 0, 'i2': 0, 'mismatches'..."
...,...,...,...,...,...
460,2fn0,A,2fn1,A,"{'length': 437, 'i1': 0, 'i2': 0, 'mismatches'..."
461,1oqe,F,1kxg,A,"{'length': 144, 'i1': 0, 'i2': 8, 'mismatches'..."
462,1xgv,A,1tyo,A,"{'length': 435, 'i1': 0, 'i2': 0, 'mismatches'..."
463,1i2h,A,1ddv,A,"{'length': 111, 'i1': 5, 'i2': 0, 'mismatches'..."


In [29]:
# pridat is_holo_paper na ty pary
# mergenout pairs...
chains2 = chains.set_index(['pdb_code', 'chain_id'])
pairs2 = pairs.merge(chains2, left_on=['pdb_code_apo', 'chain_id_apo'], right_index=True)
print(len(pairs2))
pairs2 = pairs2.merge(chains2, left_on=['pdb_code_holo', 'chain_id_holo'], right_index=True, suffixes=['_apo', '_holo'])
pairs2

465


,pdb_code_apo,chain_id_apo,pdb_code_holo,chain_id_holo,lcs_result,path_apo,is_holo_paper_apo,is_holo_apo,resolution_apo,_exptl.method_apo,...,apo_chain_id_x_holo,holo_chain_id_x_holo,apo_pdb_code_x_holo,holo_pdb_code_x_holo,domain_count_y_holo,ligand_codes_y_holo,apo_chain_id_y_holo,holo_chain_id_y_holo,apo_pdb_code_y_holo,holo_pdb_code_y_holo
0,2j1r,A,2j1s,A,"{'length': 151, 'i1': 0, 'i2': 0, 'mismatches'...",pdb_structs/2j1r.cif.gz,False,False,1.54,X-RAY DIFFRACTION,...,NaN,NaN,NaN,NaN,1.0,FUL;,A,A,2j1r,2j1s
1,2bnu,B,2bnq,E,"{'length': 241, 'i1': 0, 'i2': 0, 'mismatches'...",pdb_structs/2bnu.cif.gz,False,False,1.40,X-RAY DIFFRACTION,...,NaN,NaN,NaN,NaN,2.0,SER-LEU-LEU-MET-TRP-ILE-THR-GLN-VAL;,B,E,2bnu,2bnq
76,2bnu,B,2bnq,E,"{'length': 241, 'i1': 0, 'i2': 0, 'mismatches'...",pdb_structs/2bnu.cif.gz,False,False,1.40,X-RAY DIFFRACTION,...,NaN,NaN,NaN,NaN,2.0,SER-LEU-LEU-MET-TRP-ILE-THR-GLN-VAL;,B,E,2bnu,2bnq
2,2aq6,B,1y30,A,"{'length': 147, 'i1': 0, 'i2': 0, 'mismatches'...",pdb_structs/2aq6.cif.gz,False,False,1.70,X-RAY DIFFRACTION,...,NaN,NaN,NaN,NaN,1.0,FMN;,B,A,2aq6,1y30
3,1p5h,B,1p5r,A,"{'length': 428, 'i1': 0, 'i2': 0, 'mismatches'...",pdb_structs/1p5h.cif.gz,False,False,2.20,X-RAY DIFFRACTION,...,NaN,NaN,NaN,NaN,4.0,COA;,B,A,1p5h,1p5r
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,2fn0,A,2fn1,A,"{'length': 437, 'i1': 0, 'i2': 0, 'mismatches'...",pdb_structs/2fn0.cif.gz,False,False,1.85,X-RAY DIFFRACTION,...,NaN,NaN,NaN,NaN,3.0,SAL;,A,A,2fn0,2fn1
461,1oqe,F,1kxg,A,"{'length': 144, 'i1': 0, 'i2': 8, 'mismatches'...",pdb_structs/1oqe.cif.gz,False,False,2.50,X-RAY DIFFRACTION,...,NaN,NaN,NaN,NaN,1.0,CIT;,F,A,1oqe,1kxg
462,1xgv,A,1tyo,A,"{'length': 435, 'i1': 0, 'i2': 0, 'mismatches'...",pdb_structs/1xgv.cif.gz,False,False,2.20,X-RAY DIFFRACTION,...,NaN,NaN,NaN,NaN,3.0,ENP;,A,A,1xgv,1tyo
463,1i2h,A,1ddv,A,"{'length': 111, 'i1': 5, 'i2': 0, 'mismatches'...",pdb_structs/1i2h.cif.gz,False,False,1.80,X-RAY DIFFRACTION,...,NaN,NaN,NaN,NaN,1.0,THR-PRO-PRO-SER-PRO-PHE;,A,A,1i2h,1ddv


In [30]:
pairs = pairs2

In [36]:
# todo tohle jsem asi nespoustel??
true_positive_pair_indices = (~pairs.is_holo_paper_apo) & (pairs.is_holo_paper_holo)
true_positive_pairs = pairs[true_positive_pair_indices]
# (pro zajimavost se podivat, jesti se vytvorily nejake FP páry (dava smysl, ze ne))
print(f'false positive pairs count {len(pairs[~true_positive_pair_indices])}')

# true_positive_pairs_count = 3 + len(true_positive_pairs)  # +3 are in 4-groups, see one of the bottom cells
true_positive_pairs_count = len(true_positive_pairs)
print(f'paper pairs count {len(paper_pairs)}')
print(f'true positive pairs count {true_positive_pairs_count}')
print(f'recall {true_positive_pairs_count/ len(paper_pairs):.3f}')

print('false positive pair:')
pairs[~true_positive_pair_indices]
# a pair within the same structure. It appeared now after the fix (more strict condition on holo-ness). I don't think
# make_pairs_lcs.py changed. Take a look on the structure


false positive pairs count 1
paper pairs count 521
true positive pairs count 464
recall 0.891
false positive pair:


,pdb_code_apo,chain_id_apo,pdb_code_holo,chain_id_holo,lcs_result,path_apo,is_holo_paper_apo,is_holo_apo,resolution_apo,_exptl.method_apo,...,apo_chain_id_x_holo,holo_chain_id_x_holo,apo_pdb_code_x_holo,holo_pdb_code_x_holo,domain_count_y_holo,ligand_codes_y_holo,apo_chain_id_y_holo,holo_chain_id_y_holo,apo_pdb_code_y_holo,holo_pdb_code_y_holo
269,2g16,A,2g16,B,"{'length': 3, 'i1': 24, 'i2': 71, 'mismatches'...",pdb_structs/2g16.cif.gz,True,False,2.0,X-RAY DIFFRACTION,...,NaN,NaN,NaN,NaN,1.0,CWR;,A,B,1yhh,2g16



## Porovnat uniprot-isoform skupiny (ideálně všechny vel. 2)


In [11]:
chains2 = chains.set_index(['isoform', 'pdb_code', 'chain_id'], verify_integrity=True)
# chains[]
isoform_groups = chains2.groupby(level='isoform')
chains2['isoform_group_size'] = isoform_groups['path'].transform(len)  # todo jde nejak dostat row counts, bez toho, abych musel vybirat primo nejaky (ale libovolny) sloupec (treba `path`)
# by doing transform, I don't have to do a merge (it preserves the df len)
# apply won't work - result won't be broadcasted to original df length
isoform_group_sizes = isoform_groups.size()
print(len(isoform_group_sizes[isoform_group_sizes != 2]))
isoform_group_sizes[isoform_group_sizes != 2]


KeyError: "None of ['isoform'] are in the columns"

Three groups of 4 structures and 21 groups of one structure

In [16]:
chains2[chains2.isoform_group_size != 2]


path  \
isoform  pdb_code chain_id                                                      
Q6KB05   1mvu     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
P69178   1c48     E         apo_holo_structure_stats/paper_repl/pdb_struct...   
P69179   1cqf     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
P00766   1cgj     E         apo_holo_structure_stats/paper_repl/pdb_struct...   
         1ab9     B         apo_holo_structure_stats/paper_repl/pdb_struct...   
                  C         apo_holo_structure_stats/paper_repl/pdb_struct...   
P00257-2 1e6e     B         apo_holo_structure_stats/paper_repl/pdb_struct...   
P00257   2bt6     B         apo_holo_structure_stats/paper_repl/pdb_struct...   
O83008   1edq     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
P07254   1ffq     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
P0A8M3   1evk     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
         1evl     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
P00766   1gl1     B         apo_holo_structure_stats/paper_repl/pdb_struct...   
P37595   1k2x     D         apo_holo_structure_stats/paper_repl/pdb_struct...   
Q8GEZ8   1r9d     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
P01869   1a3l     H         apo_holo_structure_stats/paper_repl/pdb_struct...   
Q569W9   1rur     H         apo_holo_structure_stats/paper_repl/pdb_struct...   
P0A8M3   1tje     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
         1tke     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
P58162   1uc7     B         apo_holo_structure_stats/paper_repl/pdb_struct...   
P36655   2fwh     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
P09012   1nu4     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
Q9CAQ2   1xmt     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
P42212   1yhh     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
         2g16     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
                  B         apo_holo_structure_stats/paper_repl/pdb_struct...   
         1yhi     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
P03367   1ztz     B         apo_holo_structure_stats/paper_repl/pdb_struct...   
P04585   1a30     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
P62508   2gp7     B         apo_holo_structure_stats/paper_repl/pdb_struct...   
P62509   1s9p     B         apo_holo_structure_stats/paper_repl/pdb_struct...   
P15273   2i42     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
O68720   1qz0     A         apo_holo_structure_stats/paper_repl/pdb_struct...   

                            is_holo_paper  is_holo  domain_count_x  \
isoform  pdb_code chain_id                                           
Q6KB05   1mvu     A                  True     True             NaN   
P69178   1c48     E                 False    False             1.0   
P69179   1cqf     A                  True     True             NaN   
P00766   1cgj     E                 False    False             1.0   
         1ab9     B                  True     True             NaN   
                  C                  True     True             NaN   
P00257-2 1e6e     B                 False    False             1.0   
P00257   2bt6     B                  True     True             NaN   
O83008   1edq     A                 False    False             3.0   
P07254   1ffq     A                  True     True             NaN   
P0A8M3   1evk     A                 False    False             2.0   
         1evl     A                  True     True             NaN   
P00766   1gl1     B                 False    False             1.0   
P37595   1k2x     D                 False    False             1.0   
Q8GEZ8   1r9d     A                  True     True             NaN   
P01869   1a3l     H                  True     T

## Porovnat pocet paru, ktery nam vysel stejne, (zbytek budou pary, ktere se nevytvorily, bud oba apo nebo oba holo -> počty)

In [17]:
# for each group (bud size ==2 nebo >2), do A-H pairs
# > 2 muzu dostat i nejaky FP oprati paperu
# == 2 budu mit pouze missing pary (protoze oba holo/apo, nebo neuvazuju tu strukuru)
# vlastne ty skupine jeste by teoreticky mohly byt jine, nez v paperu, takze tam bych taky mohl dostat FP i FN (ale to asi nebudou)

def make_pairs(isoform_group):
    df = isoform_group
    apo = df[~df.is_holo]
    holo = df[df.is_holo]
    return apo.reset_index().merge(holo.reset_index(), how='cross', suffixes=('_apo', '_holo'))

# groups[apo] groups[holo] crossproduct
pairs = isoform_groups.apply(make_pairs).set_index(['isoform_apo', 'pdb_code_apo', 'chain_id_apo',
                                                    'isoform_holo', 'pdb_code_holo', 'chain_id_holo'],
                                                   verify_integrity=True)
pairs

,,,,,,path_apo,is_holo_paper_apo,is_holo_apo,domain_count_x_apo,ligand_codes_x_apo,apo_chain_id_x_apo,holo_chain_id_x_apo,apo_pdb_code_x_apo,holo_pdb_code_x_apo,domain_count_y_apo,...,holo_chain_id_x_holo,apo_pdb_code_x_holo,holo_pdb_code_x_holo,domain_count_y_holo,ligand_codes_y_holo,apo_chain_id_y_holo,holo_chain_id_y_holo,apo_pdb_code_y_holo,holo_pdb_code_y_holo,isoform_group_size_holo
isoform_apo,pdb_code_apo,chain_id_apo,isoform_holo,pdb_code_holo,chain_id_holo,,,,,,,,,,,,,,,,,,,,,
A0A0H2US34,2j1r,A,A0A0H2US34,2j1s,A,apo_holo_structure_stats/paper_repl/pdb_struct...,False,False,1.0,FUL;,A,A,2j1r,2j1s,NaN,...,NaN,NaN,NaN,1.0,FUL;,A,A,2j1r,2j1s,2
A0A0K0K1A5,2bnu,B,A0A0K0K1A5,2bnq,E,apo_holo_structure_stats/paper_repl/pdb_struct...,False,False,2.0,SER-LEU-LEU-MET-TRP-ILE-THR-GLN-VAL;,B,E,2bnu,2bnq,NaN,...,NaN,NaN,NaN,2.0,SER-LEU-LEU-MET-TRP-ILE-THR-GLN-VAL;,B,E,2bnu,2bnq,2
B0R5M0,2cc9,A,B0R5M0,2cc8,A,apo_holo_structure_stats/paper_repl/pdb_struct...,False,False,1.0,RBF;,A,A,2cc9,2cc8,NaN,...,NaN,NaN,NaN,1.0,RBF;,A,A,2cc9,2cc8,2
O06553,2aq6,B,O06553,1y30,A,apo_holo_structure_stats/paper_repl/pdb_struct...,False,False,1.0,FMN;,B,A,2aq6,1y30,NaN,...,NaN,NaN,NaN,1.0,FMN;,B,A,2aq6,1y30,2
O06644,1p5h,B,O06644,1p5r,A,apo_holo_structure_stats/paper_repl/pdb_struct...,False,False,4.0,COA;,B,A,1p5h,1p5r,NaN,...,NaN,NaN,NaN,4.0,COA;,B,A,1p5h,1p5r,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q9X9I8,2fn0,A,Q9X9I8,2fn1,A,apo_holo_structure_stats/paper_repl/pdb_struct...,False,False,3.0,SAL;,A,A,2fn0,2fn1,NaN,...,NaN,NaN,NaN,3.0,SAL;,A,A,2fn0,2fn1,2
Q9Y275,1oqe,F,Q9Y275,1kxg,A,apo_holo_structure_stats/paper_repl/pdb_struct...,False,False,1.0,CIT;,F,A,1oqe,1kxg,NaN,...,NaN,NaN,NaN,1.0,CIT;,F,A,1oqe,1kxg,2
Q9YE81,1xgv,A,Q9YE81,1tyo,A,apo_holo_structure_stats/paper_repl/pdb_struct...,False,False,3.0,ENP;,A,A,1xgv,1tyo,NaN,...,NaN,NaN,NaN,3.0,ENP;,A,A,1xgv,1tyo,2


##### Classification report

In [18]:
# spocitat pary
# FP pary nebudou (protoze mame dataset danej tema paper parama uz puvodne), ledaze v groups > 4
pairs2 = pairs[pairs.isoform_group_size_apo == 2]
true_positive_pair_indices = (~pairs2.is_holo_paper_apo) & (pairs2.is_holo_paper_holo)
true_positive_pairs = pairs2[true_positive_pair_indices]
# (pro zajimavost se podivat, jesti se vytvorily nejake FP páry (dava smysl, ze ne))
assert len(pairs2[~true_positive_pair_indices]) == 0

true_positive_pairs_count = 3 + len(true_positive_pairs)  # +3 are in 4-groups, see one of the bottom cells

# vyprintovat rozdil mezi pocty paru v paperu a nami nalezenych, melo by to byt max kolik procent?
print(f'paper pairs count {len(paper_pairs)}')
print(f'true positive pairs count {true_positive_pairs_count}')
print(f'recall {true_positive_pairs_count/ len(paper_pairs):.3f}')

# What is the expected recall (lower bound)?
# 1 struct was skipped due to being a protein-nucleid acid complex
# 6 structs without isoform information (api 404)
# max. 33 - 12 + 4 pairs not processed (groups size != 2)
# accuracy on is_holo 0.958 in 1035 structs
tp_pairs_lower_bound = 521 - 1 - 6 - (33-3*4) - ((1 - 0.958) * 1035)
print(f'recall expected at least {tp_pairs_lower_bound/len(paper_pairs):.3f}')

paper pairs count 521
true positive pairs count 462
recall 0.887
recall expected at least 0.863


Zbývá mi tedy na prozkoumání:
a) tři 4-groupy a
b) 33-12 (=21) 1-group

### Tři 4-groupy
P00766, P0A8M3, P42212

Zkusim dát do run_analyses ty tři 4-groupy.

Proč jim vlastně vyšly dva páry (dvě sekvence) se stejnym UNP id 2, když clusterovali na 35 % identity?
- to uvidim, až spočítám LCS snad




In [19]:
import logging
import importlib
import apo_holo_structure_stats.paper_repl.main
importlib.reload(apo_holo_structure_stats.paper_repl.main)
from apo_holo_structure_stats.pipeline.run_analyses import JSONAnalysisSerializer
from apo_holo_structure_stats.paper_repl.main import process_pair

logging.root.setLevel(logging.WARNING)

# vybrat isoform_group_size >2 a hodit je do run_analyses (process_pair)
df = pairs[pairs.isoform_group_size_apo > 2]

analyses_fname = 'output_three_large_groups.json'
analyses_serializer = JSONAnalysisSerializer(analyses_fname)
_domains_info = []

for row in df.itertuples():
    print('processing', row.Index)  # todo mel jsem udelat s reset index, ale doufal jsem, ze row.Index taky bude namedtuple
    index = row.Index  # ('P00766', '1cgj', 'E', 'P00766', '1ab9', 'B')
    process_pair(index[1], index[4], index[2], index[5], analyses_serializer, _domains_info)

# print(analyses_serializer.data)
# analyses_serializer.dump_data()


processing ('P00766', '1cgj', 'E', 'P00766', '1ab9', 'B')


processing ('P00766', '1cgj', 'E', 'P00766', '1ab9', 'C')


processing ('P00766', '1gl1', 'B', 'P00766', '1ab9', 'B')


processing ('P00766', '1gl1', 'B', 'P00766', '1ab9', 'C')


processing ('P0A8M3', '1evk', 'A', 'P0A8M3', '1evl', 'A')
processing ('P0A8M3', '1evk', 'A', 'P0A8M3', '1tke', 'A')


processing ('P0A8M3', '1tje', 'A', 'P0A8M3', '1evl', 'A')


processing ('P0A8M3', '1tje', 'A', 'P0A8M3', '1tke', 'A')
processing ('P42212', '1yhh', 'A', 'P42212', '2g16', 'A')
processing ('P42212', '1yhh', 'A', 'P42212', '2g16', 'B')


processing ('P42212', '1yhi', 'A', 'P42212', '2g16', 'A')
processing ('P42212', '1yhi', 'A', 'P42212', '2g16', 'B')


2g16B nema nic spolecnyho s 2g16A, sekvencne (jine useky jednoho UNP)
takze udelali ty LCS a pak clusterovali ty LCS samozrejmě.
Takze jim vysly dva pary, disjunktni sekvence, i kdyz ze stejnech UNP


1evkA 1evlA
1tjeA 1tkeA
- tady vyšly opravdu jenom tyto dvě kombinace
- zas asi disjunktni (overit v unp/ebi)

P00766
- divny, tady vypada, ze vsechny sekvence maj neco spolecnyho
- ledaze nejaka mutace ke konci, ale konec stejnej?
- 1cgjE 1ab9B
- 1ab9B a 1ab9C disjunktni
- to, ze jich ja mam vic nevadi, protoze oni brali jenom cluster center, ja beru kartezskej soucin

### 21 1-group

In [20]:
chains2[chains2.isoform_group_size != 2]


path  \
isoform  pdb_code chain_id                                                      
Q6KB05   1mvu     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
P69178   1c48     E         apo_holo_structure_stats/paper_repl/pdb_struct...   
P69179   1cqf     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
P00766   1cgj     E         apo_holo_structure_stats/paper_repl/pdb_struct...   
         1ab9     B         apo_holo_structure_stats/paper_repl/pdb_struct...   
                  C         apo_holo_structure_stats/paper_repl/pdb_struct...   
P00257-2 1e6e     B         apo_holo_structure_stats/paper_repl/pdb_struct...   
P00257   2bt6     B         apo_holo_structure_stats/paper_repl/pdb_struct...   
O83008   1edq     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
P07254   1ffq     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
P0A8M3   1evk     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
         1evl     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
P00766   1gl1     B         apo_holo_structure_stats/paper_repl/pdb_struct...   
P37595   1k2x     D         apo_holo_structure_stats/paper_repl/pdb_struct...   
Q8GEZ8   1r9d     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
P01869   1a3l     H         apo_holo_structure_stats/paper_repl/pdb_struct...   
Q569W9   1rur     H         apo_holo_structure_stats/paper_repl/pdb_struct...   
P0A8M3   1tje     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
         1tke     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
P58162   1uc7     B         apo_holo_structure_stats/paper_repl/pdb_struct...   
P36655   2fwh     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
P09012   1nu4     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
Q9CAQ2   1xmt     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
P42212   1yhh     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
         2g16     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
                  B         apo_holo_structure_stats/paper_repl/pdb_struct...   
         1yhi     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
P03367   1ztz     B         apo_holo_structure_stats/paper_repl/pdb_struct...   
P04585   1a30     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
P62508   2gp7     B         apo_holo_structure_stats/paper_repl/pdb_struct...   
P62509   1s9p     B         apo_holo_structure_stats/paper_repl/pdb_struct...   
P15273   2i42     A         apo_holo_structure_stats/paper_repl/pdb_struct...   
O68720   1qz0     A         apo_holo_structure_stats/paper_repl/pdb_struct...   

                            is_holo_paper  is_holo  domain_count_x  \
isoform  pdb_code chain_id                                           
Q6KB05   1mvu     A                  True     True             NaN   
P69178   1c48     E                 False    False             1.0   
P69179   1cqf     A                  True     True             NaN   
P00766   1cgj     E                 False    False             1.0   
         1ab9     B                  True     True             NaN   
                  C                  True     True             NaN   
P00257-2 1e6e     B                 False    False             1.0   
P00257   2bt6     B                  True     True             NaN   
O83008   1edq     A                 False    False             3.0   
P07254   1ffq     A                  True     True             NaN   
P0A8M3   1evk     A                 False    False             2.0   
         1evl     A                  True     True             NaN   
P00766   1gl1     B                 False    False             1.0   
P37595   1k2x     D                 False    False             1.0   
Q8GEZ8   1r9d     A                  True     True             NaN   
P01869   1a3l     H                  True     T

1-groupa
- je možné, že jde o jiný gen (UNP), ale prostě tam našli dlouhý LCS

Některé z nich jsou ty, co ztratily partnera isoform==null nebo to byl ten RNA chain (max 7)
- např 1mvu (isoform(1ap2) == null)

P69178,1c48
P69179,1cqf
- bakteriofágové ale jiní
- sekvence stejné

P00257-2,1e6e
P00257,2bt6
- nechápu, proč to SIFTS dalo to prvni na tu -2 isoformu
- alignment 1e6e sekv. a P00257 má vyšší skóre (vypadá i líp)
    - P00257-2 aligment https://www.ebi.ac.uk/Tools/services/web/toolresult.ebi?jobId=emboss_needle-I20211126-144339-0000-20069487-p2m
    - P00257 https://www.ebi.ac.uk/Tools/services/web/toolresult.ebi?jobId=emboss_needle-I20211126-144425-0497-77990938-p1m

O83008,1edq
P07254,1ffq
- oba Organism Serratia marcescens
- zas nechapu, proc to priradilo jiny isoformy (mezi sebou dost mutaci)
- pritom poly sekvence v mmcifu jsou uplne stejny
- spravna vypada O83008
- Je to stejny i na webu ebi, takze asi je problem nekde v SIFTS?
    - nevim, proc to takhle zaradil, dela to podle taxonomy + sekvence. Taxonomy je u obou pdb entry stejna, sekvence mnohem podobnejsi je ta O83008
- v mmcifu je accession 1ffq jiny! _struct_ref.pdbx_db_accession          AB015996, ale je to GB (genebank)


P01869,1a3l
Q569W9,1rur
- todo, podobne, jako dva posledni pripady?
- v mmcifu je chain 1rur H na P01869, tak proč?, API vraci presne obracene.. https://www.ebi.ac.uk/pdbe/graph-api/mappings/isoforms/1rur
...




## Závěr
Kolik jednotlivých struktur jsme "ztratili" oproti paperu a můžeme to nějak ovlivnit.

Proč API nevrací isoform (jen 6 případů)?
- zjištěno: 3 variable chain immunoglobulinů, 3 obsolete

Zjistit, proč SIFTS mapuje na horší UNP, i když má větší similarity s jinou sekvencí (a je ve stejnym taxa).
- tak +- 15 případů

IsHolo accuracy 1035 - 992 = 43 případů [cell](#is_holo_classification)
- web LPC mi nefunguje na peptid ligandy
- viz cell na apo/holo classification

## TODO

Vykašlat se na isoformy, jako jsem to udělal v production a udělat LCS within primary UNP, a pak spočítat, kolik mám párů..

Však to můžu hodit do těch skriptů, co mám, nebo ne? Jo..